In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import csv
from PIL import Image
import os
import librosa
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import cv2
import json
from keras import layers
from keras.layers import Input, Add, Dense, Activation, Dropout, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, History
from keras.utils import layer_utils
from keras.utils.np_utils import to_categorical
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import RMSprop,Adam
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications.efficientnet import preprocess_input
import pydot
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.model_selection import train_test_split
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from keras.regularizers import L1L2
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [ ]:
f = open('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json')
real_labels = json.load(f)
real_labels = {int(k):v for k,v in real_labels.items()}
real_labels

In [ ]:
train_df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train_df['class_name'] = train_df.label.map(real_labels)
train_df

In [ ]:
train,valid = train_test_split(train_df, test_size = 0.1, random_state = 42, stratify = train_df['class_name'])

In [ ]:
print(len(train))
print(len(valid))

In [ ]:
classes = len(train_df.label.unique())

In [ ]:
dataset_gen_train = ImageDataGenerator(
    preprocessing_function= preprocess_input,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest')

train_data = dataset_gen_train.flow_from_dataframe(train,
                             directory = '../input/cassava-leaf-disease-classification/train_images',
                             seed=42,
                             x_col = 'image_id',
                             y_col = 'class_name',
                             target_size = (380, 380),
                             color_mode= 'rgb',
                             class_mode = 'categorical',
                             interpolation = 'nearest',
                             shuffle = True,
                             batch_size = 20)

dataset_gen_val = ImageDataGenerator(
                    preprocessing_function = preprocess_input,
                    )

valid_data = dataset_gen_val.flow_from_dataframe(valid,
                             directory = '../input/cassava-leaf-disease-classification/train_images',
                             seed=42,
                             x_col = 'image_id',
                             y_col = 'class_name',
                             target_size = (380, 380),
                             color_mode= 'rgb',
                             class_mode = 'categorical',
                             interpolation = 'nearest',
                             shuffle = True,
                             batch_size = 20)

In [ ]:
def model_generator():
    model = Sequential()
    model.add(EfficientNetB4(include_top = False,
                         weights = '../input/tfkerasefficientnetimagenetnotop/efficientnetb4_notop.h5',
                         input_shape = (380, 380, 3),
                         drop_connect_rate = 0.4,    
                         ))

    model.add(GlobalAveragePooling2D())
    model.add(Flatten())

    model.add(Dense(512, activation = 'relu', bias_regularizer = L1L2(l1=0.01, l2=0.00)))
    model.add(Dropout(0.2))

    model.add(Dense(5, activation = 'softmax'))

    return model

training_model = model_generator()

In [ ]:
training_model.summary()

In [ ]:
SPE = train_data.n // train_data.batch_size
VS = valid_data.n // valid_data.batch_size

In [ ]:
def model_fit():
    
    optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
    
    training_model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])
    
    
    early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=3, restore_best_weights=True, verbose=1)

    checkpoint = ModelCheckpoint("Trained_Best_Model.h5", save_best_only=True, monitor = 'val_loss', mode= 'min')

    reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, patience = 2, min_lr = 1e-6, mode = 'min', verbose = 1)

    training = training_model.fit(train_data, validation_data = valid_data , epochs = 25, 
          batch_size = 20,
          steps_per_epoch = SPE,
          validation_steps = VS,                                          
          callbacks=[early_stopping, checkpoint, reduce_lr])

    training_model.save('Trained_Final_Model.h5')
    
    return training


result = model_fit()

In [ ]:
print('Training Accuracy: ' + str(max(result.history['categorical_accuracy'])))
print('Validation Accuracy: ' + str(max(result.history['val_categorical_accuracy'])))

In [ ]:
img = cv2.imread('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = resize(img, (380, 380, 3))
img = img.reshape(1, 380, 380, 3)
pred_model = keras.models.load_model('/kaggle/working/Trained_Best_Model.h5')
y = pred_model.predict(img)
y

In [ ]:
y = np.argmax(y)
y

In [ ]:
test_dir = '../input/cassava-leaf-disease-classification/test_images/'
preds = [('image_id', 'label')]
for img in os.listdir(test_dir):
    path = test_dir + img
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize(image, (380,380,3))
    image = image.reshape(1, 380, 380, 3)
    y = pred_model.predict(image)
    y = np.argmax(y)
    tup = (img, y)
    preds.append(tup)

In [ ]:
csvfile = open('submission.csv','w',newline='')
obj = csv.writer(csvfile)
for pred in preds:
    obj.writerow(pred)
csvfile.close()

In [ ]:
output1 = pd.read_csv('/kaggle/working/submission.csv')
output1